# Подключение

In [1]:
import pyodbc

In [13]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.199.13.60;DATABASE=rway;UID=vkomarnitskii;PWD=Rway1')
cursor = conn.cursor()
cursor.close()

## Pandas

In [3]:
import pandas as pd
from math import isnan

Тут можно установить параметры отображения для pandas. Сколько показывать строк и столбцов соответственно.

In [5]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 150

## Необходимые функции

Позволяет получить список характеристик для всех предложений, указанных в list_of_keys. Предложения указываются в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

Преобразование из байтов в такую строку делается следующим образом:
```python
# type(x) == bytes
result = '0x' + x.hex().upper()
```

In [93]:
def get_harks_by_object_keys(task_id):
    """
    :param task_id: id задачи
    :type task_id: str
    :return: DataFrame, в котором указаны характеристики и их значения для всех объектов из задачи task_id
    """
    
#     in_expr = f'({", ".join(list_of_keys)})'
    query = '''
    SELECT
    t_har.Наименование,
    CASE 
        WHEN t.Значение_Тип = 04 THEN CAST(t.Значение_Дата AS varchar)
        WHEN t.Значение_Тип = 03 THEN CAST(t.Значение_Число AS varchar)
        WHEN t.Значение_Тип = 05 THEN t.Значение_Строка
        WHEN t.Значение_Тип = 01 THEN 
        CASE
            WHEN t.Значение = 0x869CEA7372E5CA3E468F18025D610C0B THEN 'Да'
            WHEN t.Значение = 0xB0CD188371B9A88E431B073454A9489F THEN 'Нет'
            ELSE 'НетДанных'
        END
        WHEN t.Значение_Тип = 08 THEN 
        COALESCE(
            (SELECT Наименование FROM [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop WHERE t_dop.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types WHERE t_types.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыСделки] t_types_1 WHERE t_types_1.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ЛогическиеЗначения] t_logic WHERE t_logic.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыВерифицированности] t_verif WHERE t_verif.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ПодСегменты] t_podseg WHERE t_podseg.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[Застройщики] t_zastr WHERE t_zastr.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations WHERE t_stations.Ссылка = t.Значение)                  
        )
    END
        AS Значение,
    t.Объект AS Ссылка
FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
    INNER JOIN [rway].[ПВХ].[Характеристики] t_har
        ON t_har.[Ссылка] = t.[Характеристика]
    INNER JOIN [rway].[РегистрСведений].[ПредложенияЗадач] reg_task
        ON reg_task.Предложение = t.Объект
        AND reg_task.Задача = (SELECT _IDRRef FROM [rway].[dbo].[_Task62] WHERE _FLD198 = '{}')
    '''.format(task_id)
    
#     cursor.execute(query)
    data_df = pd.read_sql(sql=query, con=conn)

#     # Преобразование данных, которые вернул запрос в список списков, одновременно все байты преобразуются в hex
#     data = list(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), cursor.fetchall()))
#     data_df = pd.DataFrame(data, columns=list(map(lambda x: x[0], cursor.description)))
#     # Создание словаря для DataFrame
    return data_df.pivot(index='Ссылка', columns='Наименование', values='Значение')

    

Функция позволяет получить все предложения по конкретной задаче. Задача указывается в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

In [20]:
def get_base_info_by_task_id(task_id, offers_count=None):
    """
    :param task_id: id задачи
    :param offers_count: Количество предложений для выбора из базы (пустое для выбора всех предложений)
    :type task_id: str
    :return: DataFrame, содержащий все поля, которые можно получить из таблицы "ПредложенияЗадач"
    """
    
    top = '' if not offers_count else 'TOP {}'.format(offers_count)
    query = '''
    SELECT {}
          Код,
          Ссылка,
          АдресAhunter,
          АктуальнаяСсылкаИсточника,
          ДатаПересмотраЭкспозиции,
          ДатаПроверкиАктуальности,
          ДатаРазмещения,
          Город,
          Описание,
          (SELECT Наименование FROM [rway].[Справочник].[Подсегменты] WHERE Ссылка = t.Подсегмент) AS Подсегмент,
          (SELECT Наименование FROM [rway].[Справочник].[Сегменты] WHERE Ссылка = t.Сегмент) AS Сегмент,
          (SELECT Наименование FROM [rway].[Справочник].[СубъектыРФ] WHERE Ссылка = t.Субъект) AS Субъект,
          СсылкаИсточника,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыРынка] WHERE Ссылка = t.ТипРынка) AS ТипРынка,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыСделки] WHERE Ссылка = t.ТипСделки) AS ТипСделки
    FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
        JOIN [rway].[РегистрСведений].[ПредложенияЗадач] t_offer
            ON Ссылка = t_offer.[Предложение]
        JOIN [rway].[dbo].[_Task62] t_task
            ON t_task.[_IDRRef] = t_offer.[Задача] AND t_task.[_IDRRef] = (SELECT _IDRRef FROM [rway].[dbo].[_Task62] WHERE _FLD198 = '{}')

    '''.format(top, task_id)
    
#     cursor.execute(query)    
#     data = cursor.fetchall()
#     # Те же самые преобрзования, как в функции с характеристиками
#     df = pd.DataFrame(map(lambda x: list(x), data))
#     df.columns = list(map(lambda x: x[0], cursor.description))
    df = pd.read_sql(sql=query, con=conn)
    return df
    

# Боевой запуск

In [94]:
# Сбор основных данных из "ПредложенияОбъектовНедвижимости"
base_df = get_base_info_by_task_id('0001-0405-0001')

In [95]:
base_df



,Код,Ссылка,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки
0,190726W00152851,b'\x80\x00\x14\xcb3\x0f\xa3QO\x01\x11r\x19\xd4...,"обл Тульская, г Тула, пр-кт Ленина, дом 112Б",,2019-07-26 00:00:00,2019-07-26 11:46:39,2019-07-17 00:00:00,Тула,"Предлагаем в аренду помещение, под любой вид б...",ПСН,Коммерческая Недвижимость,Тульская обл,https://tula.cian.ru/rent/commercial/167207871/,Вторичный,Аренда
1,190724W00000832,b'\x80\x00h\xd70Bu\x9eD\xb0\xa5{kD\x0b\xd9',"Респ Татарстан, г Казань, ул Чистопольская, до...",,2019-07-24 00:00:00,2019-07-24 14:28:22,2019-07-22 00:00:00,Казань,Арт. 10139303 Уникальное расположение! Продаё...,офисный,Коммерческая Недвижимость,Татарстан Респ,https://kazan.cian.ru/sale/commercial/208565976/,Вторичный,Продажа
2,190725W00692765,b'\x80\x00\xb0@\x88<d\x04B!\xed\x1ey\x06@~',"обл Московская, г Химки, ул Молодежная, дом 54",,2019-07-25 00:00:00,2019-07-25 18:42:46,2019-07-25 00:00:00,Химки,Номер объекта 71024.,ПСН,Коммерческая Недвижимость,Московская обл,https://khimki.cian.ru/sale/commercial/206520716/,Вторичный,Продажа
3,190724W00022065,b'\x80\x00\xea@K\xbc/\xa5@\xdf\x95\xba\xb1\xc9...,"г Москва, ул Пироговская М., дом 14",,2019-07-24 00:00:00,2019-07-24 21:47:06,2019-07-24 00:00:00,Москва,Предлагаем в долгосрочную прямую аренду офисно...,офисный,Коммерческая Недвижимость,Москва г,https://www.cian.ru/rent/commercial/212292251/,Вторичный,Аренда
4,190724W00014544,b'\x80\x00\xfe\xde}\xdc_*M(\x90\xe6\xf8y\x8fu',"обл Кировская, г Киров, ул Заводская, дом 17",,2019-07-24 00:00:00,2019-07-24 19:21:02,2019-07-24 00:00:00,Киров,Арт. 8479574 Продается оздоровительный комплек...,гостиничный,Коммерческая Недвижимость,Кировская обл,https://kirov.cian.ru/sale/commercial/212003814/,Вторичный,Продажа
5,190726W00004545,b'\x80\x01T\x83\xcaFq\xdbNS\xef\x1a\xdd\xfe\x8...,"г Москва, ул Яблочкова, дом 21, корп 3",,2019-07-26 00:00:00,2019-07-26 00:10:05,2019-07-24 00:00:00,Москва,Сдается офисное с мебелью в пешей доступности ...,офисный,Коммерческая Недвижимость,Москва г,https://www.cian.ru/rent/commercial/207288709/,Вторичный,Аренда
6,190725W00293416,b'\x80\x01c\xb6\xf6\x94\xd4\xb7K\xb3\xab\x9b\x...,"г Москва, пер Волков, дом 7-9, стр 2",,2019-07-25 00:00:00,2019-07-25 16:48:29,2019-07-25 00:00:00,Москва,Готовый арендный бизнес. Помещение свободного ...,ПСН,Коммерческая Недвижимость,Москва г,https://www.cian.ru/sale/commercial/209554117/,Вторичный,Продажа
7,190725W00013365,"b'\x80\x01~""\x1cd\xd5\x16C\xab)\xbb\xb2G\xe2\xe1'","Респ Башкортостан, г Уфа, ул Менделеева, дом 221",,2019-07-25 00:00:00,2019-07-25 10:57:38,2019-07-24 00:00:00,Уфа,= ОТ СОБСТВЕННИКА. БЕЗ КОМИССИИ = Сдается офис...,офисный,Коммерческая Недвижимость,Башкортостан Респ,https://ufa.cian.ru/rent/commercial/205017169/,Вторичный,Аренда
8,190725W00016745,b'\x80\x01\xbf\x97\xa5\xb0#\xc4O\x84\xd2v)\xc9...,"обл Тюменская, г Тюмень, ул Дзержинского, дом ...",,2019-07-25 00:00:00,2019-07-25 13:36:17,2019-07-25 00:00:00,Тюмень,Сдается в аренду от собственника! Без комиссии...,офисный,Коммерческая Недвижимость,Тюменская обл,https://tyumen.cian.ru/rent/commercial/210943889/,Вторичный,Аренда
9,190724W00015278,b'\x80\x01\xe9-\xb52\x1f\xdfG!\x8a\xbc\xcfH\xa...,"г Москва, ул Семёновская Б., дом 40",,2019-07-24 00:00:00,2019-07-24 19:41:21,2019-07-24 00:00:00,Москва,!ПРЯМАЯ АРЕНДА ОТ СОБСТВЕННИКА без комиссии и ...,офисный,Коммерческая Недвижимость,Москва г,https://www.cian.ru/rent/commercial/212883225/,Вторичный,Аренда


In [96]:
# Сбор характеристик по ссылкам из основного DataFrame base_df
har_df = get_harks_by_object_keys('0001-0405-0001')

In [97]:
har_df

Наименование,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Класс Объекта,Количество Этажей,Кондиционирование,Материал Наружных Стен,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Этаж
Ссылка,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b'\x80\x00\x14\xcb3\x0f\xa3QO\x01\x11r\x19\xd4\x8f\x16',NaN,NaN,Ленина пр-кт,дом № 112Б,NaN,None,0.000000,37.581900,54.154391,0.000000,2019-07-26 11:46:39,"Свободное назначение, 74 м2",НетДанных,9.000000,None,None,0.000000,Включая,свободное,None,74.000000,,Отдельный Вход,0.000000,None,74.000000,0.000000,None,NaN,NaN,Сергей Парфенов,кв.м.,руб./мес.,0.000000,None,Без Проведения Торгов,Действующий,NaN,(906) 5310114,None,None,жилой дом,,58978.000000,797.000000,
b'\x80\x00h\xd70Bu\x9eD\xb0\xa5{kD\x0b\xd9',NaN,NaN,Чистопольская ул,дом № 9Б,NaN,None,2.700000,49.106585,55.820314,0.000000,2019-07-24 14:28:22,"Здание, 756 м2",НетДанных,0.000000,None,None,0.000000,Включая,офис,None,756.000000,,None,0.000000,None,756.000000,0.000000,None,NaN,NaN,"Алексей Мага, Цн1",кв.м.,НетДанных,0.000000,None,Без Проведения Торгов,None,NaN,(987) 2930768,None,None,None,,85000000.000000,112433.862434,0
b'\x80\x00\xb0@\x88<d\x04B!\xed\x1ey\x06@~',NaN,NaN,Молодежная ул,дом № 54,NaN,None,0.000000,37.399102,55.902353,0.000000,2019-07-25 18:42:46,"Свободное назначение, 116 м2",НетДанных,18.000000,None,Железобетонные Панели (С Отделкой Или Без),0.000000,УСН,свободное,None,116.000000,,None,0.000000,None,116.000000,0.000000,None,NaN,NaN,"Светлана Кузьмина, Century 21 Street Realty",кв.м.,руб.,0.000000,None,Без Проведения Торгов,None,NaN,(905) 5666602,None,None,жилой дом,,16500000.000000,142241.379310,1
b'\x80\x00\xea@K\xbc/\xa5@\xdf\x95\xba\xb1\xc9E\x18',NaN,NaN,Пироговская М. ул,дом № 14,NaN,None,0.000000,0.000000,0.000000,1990.000000,2019-07-24 21:47:06,"Офис (B+), от 150 до 306 м2",НетДанных,6.000000,None,None,0.000000,Включая,офис,Чистовая,306.000000,,Нет,40.000000,Открытая,306.000000,400.000000,Спринклерная Система,NaN,NaN,Fortex Consulting Group,кв.м.,руб./мес.,0.000000,Приточно-Вытяжная,Без Проведения Торгов,Действующий,NaN,(916) 2411956,None,Наземная,бизнес-центр,,734400.000000,2400.000000,6
b'\x80\x00\xfe\xde}\xdc_*M(\x90\xe6\xf8y\x8fu',NaN,NaN,Заводская ул,дом № 17,NaN,None,0.000000,49.675640,58.624195,0.000000,2019-07-24 19:21:02,"Офис (А), 186 м2",НетДанных,2.000000,None,None,0.000000,Включая,сауна,None,186.000000,,None,0.000000,Смешанная,186.000000,0.000000,None,NaN,NaN,"Денис Еремин, Руснедвижимость",кв.м.,НетДанных,0.000000,None,Без Проведения Торгов,None,NaN,(982) 8102176,None,None,None,,2500000.000000,13440.860215,2
b'\x80\x01T\x83\xcaFq\xdbNS\xef\x1a\xdd\xfe\x88\xa5',NaN,NaN,Яблочкова ул,"дом № 21, корпус 3",NaN,None,3.400000,0.000000,0.000000,2009.000000,2019-07-26 00:10:05,"Офис (B+), 167 м2",B+,8.000000,None,None,0.000000,УСН,офис,Чистовая,167.000000,,Нет,94.000000,Смешанная,167.000000,1300.000000,Спринклерная Система,NaN,NaN,Ндв - Cупермаркет Недвижимости,кв.м.,руб./мес.,0.000000,Приточно-Вытяжная,Без Проведения Торгов,Действующий,NaN,(985) 5203450,None,Подземная,бизнес-центр,,243541.000000,1458.329341,4
b'\x80\x01c\xb6\xf6\x94\xd4\xb7K\xb3\xab\x9b\x90\x03\xe3\xb6',NaN,NaN,Волков пер,"дом № 7-9, строение 2",NaN,None,4.000000,37.573474,55.762627,1931.000000,2019-07-25 16:48:29,"Свободное назначение, 106 м2",НетДанных,5.000000,N

In [102]:
# Собираем обе таблицы в одну большую
result_df = pd.merge(base_df, har_df, on='Ссылка', how='left')

In [103]:
result_df

,Код,Ссылка,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Класс Объекта,Количество Этажей,Кондиционирование,Материал Наружных Стен,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Этаж
0,190726W00152851,b'\x80\x00\x14\xcb3\x0f\xa3QO\x01\x11r\x19\xd4...,"обл Тульская, г Тула, пр-кт Ленина, дом 112Б",,2019-07-26 00:00:00,2019-07-26 11:46:39,2019-07-17 00:00:00,Тула,"Предлагаем в аренду помещение, под любой вид б...",ПСН,Коммерческая Недвижимость,Тульская обл,https://tula.cian.ru/rent/commercial/167207871/,Вторичный,Аренда,NaN,NaN,Ленина пр-кт,дом № 112Б,NaN,None,0.000000,37.581900,54.154391,0.000000,2019-07-26 11:46:39,"Свободное назначение, 74 м2",НетДанных,9.000000,None,None,0.000000,Включая,свободное,None,74.000000,,Отдельный Вход,0.000000,None,74.000000,0.000000,None,NaN,NaN,Сергей Парфенов,кв.м.,руб./мес.,0.000000,None,Без Проведения Торгов,Действующий,NaN,(906) 5310114,None,None,жилой дом,,58978.000000,797.000000,
1,190724W00000832,b'\x80\x00h\xd70Bu\x9eD\xb0\xa5{kD\x0b\xd9',"Респ Татарстан, г Казань, ул Чистопольская, до...",,2019-07-24 00:00:00,2019-07-24 14:28:22,2019-07-22 00:00:00,Казань,Арт. 10139303 Уникальное расположение! Продаё...,офисный,Коммерческая Недвижимость,Татарстан Респ,https://kazan.cian.ru/sale/commercial/208565976/,Вторичный,Продажа,NaN,NaN,Чистопольская ул,дом № 9Б,NaN,None,2.700000,49.106585,55.820314,0.000000,2019-07-24 14:28:22,"Здание, 756 м2",НетДанных,0.000000,None,None,0.000000,Включая,офис,None,756.000000,,None,0.000000,None,756.000000,0.000000,None,NaN,NaN,"Алексей Мага, Цн1",кв.м.,НетДанных,0.000000,None,Без Проведения Торгов,None,NaN,(987) 2930768,None,None,None,,85000000.000000,112433.862434,0
2,190725W00692765,b'\x80\x00\xb0@\x88<d\x04B!\xed\x1ey\x06@~',"обл Московская, г Химки, ул Молодежная, дом 54",,2019-07-25 00:00:00,2019-07-25 18:42:46,2019-07-25 00:00:00,Химки,Номер объекта 71024.,ПСН,Коммерческая Недвижимость,Московская обл,https://khimki.cian.ru/sale/commercial/206520716/,Вторичный,Продажа,NaN,NaN,Молодежная ул,дом № 54,NaN,None,0.000000,37.399102,55.902353,0.000000,2019-07-25 18:42:46,"Свободное назначение, 116 м2",НетДанных,18.000000,None,Железобетонные Панели (С Отделкой Или Без),0.000000,УСН,свободное,None,116.000000,,None,0.000000,None,116.000000,0.000000,None,NaN,NaN,"Светлана Кузьмина, Century 21 Street Realty",кв.м.,руб.,0.000000,None,Без Проведения Торгов,None,NaN,(905) 5666602,None,None,жилой дом,,16500000.000000,142241.379310,1
3,190724W00022065,b'\x80\x00\xea@K\xbc/\xa5@\xdf\x95\xba\xb1\xc9...,"г Москва, ул Пироговская М., дом 14",,2019-07-24 00:00:00,2019-07-24 21:47:06,2019-07-24 00:00:00,Москва,Предлагаем в долгосрочную прямую аренду офисно...,офисный,Коммерческая Недвижимость,Москва г,https://www.cian.ru/rent/commercial/212292251/,Вторичный,Аренда,NaN,NaN,Пироговская М. ул,дом № 14,NaN,None,0.000000,0.000000,0.000000,1990.000000,2019-07-24 21:47:06,"Офис (B+), от 150 до 306 м2",НетДанных,6.000000,None,None,0.000000,Включая,офис,Чистовая,306.000000,,Нет,40.000000,Открытая,306.000000,400.000000,Спринклерная Система,NaN,NaN,Fortex Consulting Group,кв.м.,руб./мес.,0.000000,Приточно-Вытяжная,Без Проведения То

Функция, в которой и происходит тестирование.

In [100]:
def make_test():
    """
    :return: DataFrame, содержащий статистику по всем задачам
    """
    test_df = pd.DataFrame(index=result_df.columns)
    test_df['Рейтинг заполненности'] = [0] * len(test_df)
    test_df['Заполнено'] = [0] * len(test_df)
    test_df['Всего'] = [len(result_df)] * len(test_df)
    
    float_null = '0.000000'
    nan = str(float('nan'))
    
    for row in result_df:
        for cell in result_df.loc[:, row]:
            if cell and not cell == float_null and cell != nan and cell != 'НетДанных':  # Проверка на неподходящие значения
                test_df.loc[row, 'Заполнено'] += 1
                
    for row in test_df.iterrows():
        test_df.loc[row[0], 'Рейтинг заполненности'] = str(row[1]['Заполнено'] * 100 // row[1]['Всего']) + '%'
        
    return test_df

In [101]:
test = make_test()

KeyboardInterrupt: 

Выгрузка данных в `csv` файл для удобного просмотра в Excel

In [ ]:
test.to_csv('test_result.csv', header=list(test.columns), encoding='cp1251', sep=';')
test

## Для тестов


In [159]:
cursor = conn.cursor()
query = '''
/****** Скрипт для команды SelectTopNRows из среды SSMS  ******/
SELECT
    task._IDRRef AS Ссылка,
    types.Наименование AS ТипЗадачи,
    task._Fld198 AS Код,
    *
  FROM [rway].[dbo].[_Task62] task
  JOIN [rway].[Документ].[Задание] tasks
      ON task._Fld192RRef = tasks.Ссылка
  JOIN [rway].[Справочник].[ТипыЗадач] types
      ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
  WHERE tasks.КодЗадания = '0001-0405'
'''
# WHERE _FLD198 = '0001-0402-0001'

cursor.execute(query)
data = cursor.fetchall()
df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
df.columns = list(map(lambda x: x[0], cursor.description))
df


,Ссылка,ТипЗадачи,Код,_IDRRef,_Version,_Marked,_BusinessProcessRRef,_PointRRef,_Date_Time,_Number,_Name,_Executed,_Fld183,_Fld184,_Fld185,_Fld2015RRef,_Fld186RRef,_Fld187,_Fld188,_Fld189,_Fld2011,_Fld190,_Fld191,_Fld192RRef,_Fld193RRef,_Fld194RRef,_Fld195,_Fld196,_Fld197RRef,_Fld198,_Fld199,_Fld200,_Fld201,_Fld202,_Fld203,_Fld204,_Fld205,_Fld206,_Fld207,_Fld208,_Fld209,_Fld210RRef,_Fld211RRef,_Fld212RRef,_Fld213RRef,_Fld214RRef,_Fld215RRef,_Fld216,_Fld217RRef,_Fld218,_Fld219,_Fld220,_Fld221,_Fld222,_Fld223RRef,_Fld224,_Fld225RRef,_Fld226RRef,_Fld227,_Fld228RRef,_Fld229,_Fld230,_Fld231RRef,_Fld232,_Fld233RRef,_Fld234RRef,_Fld235RRef,_Fld236RRef,_Fld237_TYPE,_Fld237_RTRef,_Fld237_RRRef,Ссылка,ВерсияДанных,ПометкаУдаления,Дата,Номер,Проведен,ДатаЗавершенияЗадания,ДатаНачалаЗадания,ЖелаемоеКоличество,КодЗадания,КоличествоИсточники,КоличествоСегменты,КоличествоСубъекты,КритерийВерификацииДляЭкспорта,Наименование,Ответственный,ОчиститьНеверифицированные,ПредметОбработки,ПредыдущееЗадание,Проект,Состояние,Статус,ТекстУсловияВерификацииSQL,УдалитьЗадание,ШаблонСценария,ШаблонУровнейВерификации,ШаблонХарактеристик,ЭкспортироватьВАрхивПолучатель,ЭкспортироватьВАрхив,ВерсияARH,Ссылка,ВерсияДанных,ПометкаУдаления,ИмяПредопределенныхДанных,Код,Наименование,НаименованиеВызова,ПоказыватьДубли,ПоказыватьСозданныеЗадачи,РучнаяВерификация,ОграничениеЗадач,ИмяФункцииОбработки,ПроизвольнаяКоллекция,ВерсияARH
0,0xBBBA30E1716029C711E9AE0130754942,Импорт,0001-0405-0001,0xBBBA30E1716029C711E9AE0130754942,0x0000000003B22DAC,0x00,0x00000000000000000000000000000000,0x00000000000000000000000000000000,4019-07-24 13:52:55,000002556,Импорт из ПСОД №0001-0405-0001 от 24.07.2019 1...,0x00,0,0,0,0x87C6B9110031EDD7474BF045B2151A49,0x00000000000000000000000000000000,2001-01-01 00:00:00,4019-07-17 00:00:00,2001-01-01 00:00:00,0,2001-01-01 00:00:00,0x00,0xBBBA30E1716029C711E9AE011C42A8FB,0x00000000000000000000000000000000,0x00000000000000000000000000000000,0x00,0x00,0x80BC00155D645E0311E7139990713D10,0001-0405-0001,1,0,0,"<ValueListType xmlns=""http://v8.1c.ru/8.1/data...",0x00,"<Settings xmlns=""http://v8.1c.ru/8.1/data-comp...","<Structure xmlns=""http://v8.1c.ru/8.1/data/cor...","<DataCompositionSchema xmlns=""http://v8.1c.ru/...",ВЫБРАТЬ РАЗРЕШЕННЫЕ РАЗЛИЧНЫЕ\n\tПредлЗаданий....,"<Array xmlns=""http://v8.1c.ru/8.1/data/core"" x...",,0x00000000000000000000000000000000,0x00000000000000000000000000000000,0x00000000000000000000000000000000,0x00000000000000000000000000000000,0xAB47107B44B1669011E996707E69FC46,0x00000000000000000000000000000000,1,0x00000000000000000000000000000000,СохранятьВарианты,,,0x01010800000000000000EFBBBF7B2255227D,,0xAB47107B44B1669011E990CEFFC204E0,0,0xAB47107B44B1669011E990D14C17FFEE,0xAB47107B44B1669011E990CEFFC204D6,,0x00000000000000000000000000000000,2001-01-01 00:00:00,"{\r\n""_start_urls"": [\r\n""<auto_start_urls>""\r...",0xAB47107B44B1669011E990CEFFC204B6,0x00,0x00000000000000000000000000000000,0x00000000000000000000000000000000,0xAB47107B44B1669011E99670D0185967,0x00000000000000000000000000000000,0x08,0x00000067,0x80C0E0071BF5BDE311E741EAD34A9F3F,0xBBBA30E1716029C711E9AE011C42A8FB,0x00000000043B7B11,0,2019-07-24 13:52:21,23,0,2019-07-31 00:00:00,2019-07-24 00:00:00,300000,0001-0405,73,1,93,0xBB9A30E1716029C711E955ECE8AA31D4,коммерческая вторичная к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0xBBBA30E1716029C711E9A85EDF78B9F2,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8A2B026DDB6080A5485EA0659DE117D3,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0xBBB030E1716029C711E9971813456627,0x80FB30E1716029C711E8E35B7F26B37A,0xBBAC30E1716029C711E9928D38D72A19,ROL Эксплуатация,1,0,0xAB47107B44B1669011E990CEFFC204B6,0x000000000001AEC5,0,0xBD2AFDC55B7F639F4FCC439E52C2B9C7,1,Импорт,,0,0,0,0,,0,0
1,0xBBBA30E1716029C711E9AEC8F29AE7E1,Импорт,0001-0405-0005,0xBBBA30E1716029C711E9AEC8F29AE7E1,0x0000000003B39270,0x00,0x00000000000000000000000000000000,0x00000000000000000000000000000

In [141]:
cursor = conn.cursor()
query = '''
/****** Скрипт для команды SelectTopNRows из среды SSMS  ******/
SELECT TOP 1000 [Ссылка]
      ,[ВерсияДанных]
      ,[ПометкаУдаления]
      ,[Дата]
      ,[Номер]
      ,[Проведен]
      ,[ДатаЗавершенияЗадания]
      ,[ДатаНачалаЗадания]
      ,[ЖелаемоеКоличество]
      ,[КодЗадания]
      ,[КоличествоИсточники]
      ,[КоличествоСегменты]
      ,[КоличествоСубъекты]
      ,[КритерийВерификацииДляЭкспорта]
      ,[Наименование]
      ,[Ответственный]
      ,[ОчиститьНеверифицированные]
      ,[ПредметОбработки]
      ,[ПредыдущееЗадание]
      ,[Проект]
      ,[Состояние]
      ,[Статус]
      ,[ТекстУсловияВерификацииSQL]
      ,[УдалитьЗадание]
      ,[ШаблонСценария]
      ,[ШаблонУровнейВерификации]
      ,[ШаблонХарактеристик]
      ,[ЭкспортироватьВАрхивПолучатель]
      ,[ЭкспортироватьВАрхив]
      ,[ВерсияARH]
  FROM [rway].[Документ].[Задание]
'''
# WHERE _FLD198 = '0001-0402-0001'

cursor.execute(query)
data = cursor.fetchall()
new_df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
new_df.columns = list(map(lambda x: x[0], cursor.description))
new_df

,Ссылка,ВерсияДанных,ПометкаУдаления,Дата,Номер,Проведен,ДатаЗавершенияЗадания,ДатаНачалаЗадания,ЖелаемоеКоличество,КодЗадания,КоличествоИсточники,КоличествоСегменты,КоличествоСубъекты,КритерийВерификацииДляЭкспорта,Наименование,Ответственный,ОчиститьНеверифицированные,ПредметОбработки,ПредыдущееЗадание,Проект,Состояние,Статус,ТекстУсловияВерификацииSQL,УдалитьЗадание,ШаблонСценария,ШаблонУровнейВерификации,ШаблонХарактеристик,ЭкспортироватьВАрхивПолучатель,ЭкспортироватьВАрхив,ВерсияARH
0,0xBBBA30E1716029C711E9AE011C42A8FB,0x00000000043B7B11,0,2019-07-24 13:52:21,23,0,2019-07-31 00:00:00,2019-07-24 00:00:00,300000,0001-0405,73,1,93,0xBB9A30E1716029C711E955ECE8AA31D4,коммерческая вторичная к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0xBBBA30E1716029C711E9A85EDF78B9F2,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8A2B026DDB6080A5485EA0659DE117D3,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0xBBB030E1716029C711E9971813456627,0x80FB30E1716029C711E8E35B7F26B37A,0xBBAC30E1716029C711E9928D38D72A19,ROL Эксплуатация,1,0
1,0xBBBA30E1716029C711E9AEDA87E5C5DD,0x00000000042BDE5E,0,2019-07-25 15:48:42,24,0,2019-07-31 00:00:00,2019-07-25 00:00:00,20000,0001-0406,76,1,85,0xBB9A30E1716029C711E955ECE8AA31D4,зу вторичная к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0xBBBA30E1716029C711E9A85ED736875B,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8A2B026DDB6080A5485EA0659DE117D3,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0xBB9A30E1716029C711E926D5CC8B7D94,0xBB9A30E1716029C711E9372CE9270C58,0xBB9A30E1716029C711E95C32BD197E1F,ROL Archive,0,0
2,0xBBBB30E1716029C711E9B133CD754E42,0x0000000004331601,0,2019-07-28 15:32:54,25,0,2019-07-31 00:00:00,2019-07-28 00:00:00,0,0001-0407,76,1,85,0xBB9A30E1716029C711E955ECE8AA31D4,готовый бизнес к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0x00000000000000000000000000000000,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8ED2D56DC8116F1349D0AD43595892BB,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0xBBB530E1716029C711E99B21F22F0E2C,0xBB9A30E1716029C711E938446B8C6EC2,0xBBB030E1716029C711E9963FA27FB1FB,ROL Archive,1,0
3,0xBBBB30E1716029C711E9B14F4BDE76D6,0x00000000043315FC,0,2019-07-28 18:49:35,26,0,2019-07-31 00:00:00,2019-07-28 00:00:00,2500,0001-0408,1,1,85,0xBB9A30E1716029C711E955ECE8AA31D4,коммерческая торги к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0x00000000000000000000000000000000,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8ED2D56DC8116F1349D0AD43595892BB,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0x80DD30E1716029C711E7CB78B41BB757,0x80F630E1716029C711E8E1B589261D9C,0xBB9A30E1716029C711E96683AF339C27,ROL Archive,1,0
4,0xBBBB30E1716029C711E9B14F5CD18A54,0x00000000042A7814,0,2019-07-28 18:50:03,27,0,2019-07-31 00:00:00,2019-07-28 00:00:00,500,0001-0409,1,0,85,0xBB9A30E1716029C711E955ECE8AA31D4,зу торги к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0x00000000000000000000000000000000,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8A2B026DDB6080A5485EA0659DE117D3,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0x80F630E1716029C711E8B66229012248,0xBB9A30E1716029C711E935097625DB69,0xBBAC30E1716029C711E99289027F858B,ROL Archive,0,0
